## What do we know about COVID-19 risk factors?	
## Data on potential risk factors: Smoking, pre-existing pulmonary disease

In [151]:
root = '/Users/u6066091/Desktop/kaggle'

In [3]:
import pandas as pd

In [18]:
terms = pd.read_csv(root + '/input/corona_challenge/COVID-19_dictionary_sub_question.csv')
risk_factors = pd.read_csv(root + '/input/corona_challenge/COVID-19 medical_dictionary-difference.dictionary.csv')

In [13]:
terms.head()

,Task,Question,Subquestion,Outcome.list,Differences.list,Design.list,Notes
0,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,Range of incubation periods for humans: genera...,incubation period,age.,"contact tracing, survival analysis, time-to-ev...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
1,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,Range of incubation periods for humans: by age...,"incubation period, stratified by age group",age.,"contact tracing, survival analysis, time-to-ev...",Notes
2,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,Range of incubation periods for humans: by hea...,"incubation period, stratified by health status...","health status (diabetes, hypertension, heart d...","contact tracing, survival analysis, time-to-ev...",Notes
3,"What is known about transmission, incubation, ...",Range of incubation periods for the disease in...,"How long are individuals contagious, even afte...","infectious period, serial interval, generation...","age, health status (diabetes, hypertension, he...",excretion profiles from experimental infection...,https://idpjournal.biomedcentral.com/articles/...
4,"What is known about transmission, incubation, ...",Prevalence of asymptomatic shedding and transm...,Prevalence of asymptomatic shedding: general p...,"nasal carriage, asymptomatic shedding","age, health status (diabetes, hypertension, he...",population prevalence survey using nasopharyng...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...


In [123]:
risk_eval = risk_factors['difference.variable']

In [178]:
outcome_list = ['Odds Ratio for COVID-19 carriage', 'Risk Ratio for COVID-19 acquisition', 
               'Hazard Ratio for COVID-19 acquisition', 'COVID-19 incidence (stratified by asthma severity)', 
               'frequency (of asthma) among COVID-19 cases in a case series', 
               'frequency (of asthma) in a COVID-19 prevalence survey', 
               'frequency (of asthma) in a COVID-19 syndromic surveillance survey']

Difference_list = ['asthma', 'asthma severity (intermittent/persistent, mild/moderate/severe)', 
                    'FEV', 'oral systemic corticosteroids']

# Design_list = ['risk factor analysis', 'logistic regression', 'retrospective cohort', 'cross-sectional case-control',
#                'matched case-control', 'medical records review', 'mortality records review', 'observational case series', 
#                'time series analysis', 'survival analysis']

Design_list = [ 'retrospective cohort', 'cross-sectional case-control','prevalence survey', 'systematic review , meta-analysis',
               'matched case-control', 'medical records review', 'observational case series', 
               'time series analysis','pseudo-randomized controlled trials' ,'randomized controlled trials']

# cov_list = ['covid',
#                     'coronavirus disease 19',
#                     'sars cov 2', # Note that search function replaces '-' with ' '
#                     '2019 ncov',
#                     '2019ncov',
#                     r'2019 n cov\b',
#                     r'2019n cov\b',
#                     'ncov 2019',
#                     r'\bn cov 2019',
#                     'coronavirus 2019',
#                     'wuhan pneumonia',
#                     'wuhan virus',
#                     'wuhan coronavirus',
#                     r'coronavirus 2\b']
cov_list = ['COVID-19', 'novel coronavirus','CORD-19' ,'2019-nCoV', 'Wuhan coronavirus']


In [15]:
new_data = pd.read_csv(root + '/output/corona_challenge/all_data_metadata.csv')

In [32]:
new_data.columns

Index(['Unnamed: 0', 'doc_id', 'source', 'title', 'abstract_x', 'text_body',
       'sha', 'source_x', 'doi', 'pmcid', 'pubmed_id', 'license', 'abstract_y',
       'publish_time', 'authors', 'journal', 'Microsoft Academic Paper ID',
       'WHO #Covidence', 'has_full_text', 'full_text_file'],
      dtype='object')

## Covid related papers

In [106]:
import re
title_cov = []

# for cov in cov_list:
for i in range(0,new_data.shape[0]):
    a =  [re.findall(cov.lower(), new_data.loc[i,'abstract_x'].lower()) for cov in cov_list]
    for aa in a:
        if aa != []:
            title_cov.append([new_data.loc[i,'title'],new_data.loc[i,'abstract_x'], new_data.loc[i,'text_body']])

In [130]:
corona_data = pd.DataFrame(title_cov, columns = ['title', 'abstract', 'text_body']).drop_duplicates('title').reset_index()


In [138]:
risk_data =[]
design_data = []

for i in range(0,corona_data.shape[0]):
    a_risk =  [re.findall(risk.lower(), corona_data.loc[i,'text_body'].lower()) for risk in risk_eval]
    a_design = [re.findall(des.lower(), corona_data.loc[i,'text_body'].lower()) for des in Design_list]

    for risk in a_risk:
        if risk != []:
            risk_data.append([corona_data.loc[i,'title'],corona_data.loc[i,'abstract'], corona_data.loc[i,'text_body']])
    for des in a_design:
        if des != []:
            design_data.append([corona_data.loc[i,'title'],corona_data.loc[i,'abstract'], corona_data.loc[i,'text_body']])


In [143]:
design_data = pd.DataFrame(design_data, columns = ['title', 'abstract', 'text_body']).drop_duplicates('title').reset_index()


In [144]:
risk_data = pd.DataFrame(risk_data, columns = ['title', 'abstract', 'text_body']).drop_duplicates('title').reset_index()


In [179]:
Design_list[0].lower()

'retrospective cohort'

In [182]:
a_risk = []
a_design = []
def token_sliding_window(str, size):
    tokens = str.lower().split(' ')
    for i in range(len(tokens )- size + 1):
        yield tokens[i: i+size]
for i in range(0,corona_data.shape[0]):
    doc = corona_data.loc[i,'text_body'].lower()
    for x in range(len(doc)-100):
        words = doc[x:x+100]
    
        a_design.append([re.findall(des.lower(), words) for des in Design_list])
 


In [ ]:
pattern =  "([^"]*(commonly used|the stability)[^"]*)((?!"section").)*"section": "([^"]*)"
                                                                                    

In [163]:
import textwrap
wrapper = textwrap.TextWrapper(width = 100)
text = risk_data.loc[7,'text_body']    
print(wrapper.fill(text))

On December 31, 2019, a cluster of cases of "pneumonia of unknown origin" in people associated with
the Huanan Seafood Wholesale Market has been reported in Wuhan, China. Only a few days later,
Chinese health authorities confirmed that this cluster was associated with a Novel Coronavirus 1 of
them had comorbidities. Importantly, the patients admitted to the intensive care unit (ICU) had a
higher number of comorbidities(72.2%) than those not admitted to the ICU(37.3%). This suggests that
complications may be a risk factor for adverse outcomes 4 . Assessing the prevalence of these
chronic diseases is the basis for mitigating complications in patients with COVID-19 infections.
However, the effort was hampered by the limited number of cases.  To get a more convincing result,
we will provide a systematic evaluation and detail not only estimate the Prevalence of comorbidities
in all patients, also assess the risk of underlying diseases in severe patients compared to non-
severe patients. The